In [1]:
import tensorflow as tf
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)
        

42


In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [7]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [8]:
housing = fetch_california_housing()
m, n = housing.data.shape
print(m, n)

20640 8


In [12]:
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]
print(housing_data_plus_bias.shape)
print(housing.target.shape)
print(housing.target.reshape(-1,1).shape)

(20640, 9)
(20640,)
(20640, 1)


In [14]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)
print(theta)

Tensor("MatMul_5:0", shape=(9, 1), dtype=float32)


In [15]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [16]:
print(theta_value)

[[-3.7465141e+01]
 [ 4.3573415e-01]
 [ 9.3382923e-03]
 [-1.0662201e-01]
 [ 6.4410698e-01]
 [-4.2513184e-06]
 [-3.7732250e-03]
 [-4.2664889e-01]
 [-4.4051403e-01]]


In [20]:
# manually compute gradients
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)


n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name='theta')
y_predict = tf.matmul(X, theta, name='predictions')
error = y_predict - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m*tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print('Epoch:', epoch, 'MSE:', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()



Epoch: 0 MSE: 3.737157
Epoch: 100 MSE: 0.67239517
Epoch: 200 MSE: 0.59016126
Epoch: 300 MSE: 0.57072186
Epoch: 400 MSE: 0.5578689
Epoch: 500 MSE: 0.54863214
Epoch: 600 MSE: 0.54195374
Epoch: 700 MSE: 0.5371207
Epoch: 800 MSE: 0.5336216
Epoch: 900 MSE: 0.5310853


In [21]:
# autodiff
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)


n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name='theta')
y_predict = tf.matmul(X, theta, name='predictions')
error = y_predict - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print('Epoch:', epoch, 'MSE:', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()


Epoch: 0 MSE: 6.5575347
Epoch: 100 MSE: 0.73061836
Epoch: 200 MSE: 0.59361374
Epoch: 300 MSE: 0.5713964
Epoch: 400 MSE: 0.5584493
Epoch: 500 MSE: 0.5492318
Epoch: 600 MSE: 0.5425448
Epoch: 700 MSE: 0.53767854
Epoch: 800 MSE: 0.53413355
Epoch: 900 MSE: 0.53154665


In [24]:
# optimizer
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)


n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name='theta')
y_predict = tf.matmul(X, theta, name='predictions')
error = y_predict - y
mse = tf.reduce_mean(tf.square(error), name='mse')

# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print('Epoch:', epoch, 'MSE:', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch: 0 MSE: 11.34516
Epoch: 100 MSE: 0.52626836
Epoch: 200 MSE: 0.52442175
Epoch: 300 MSE: 0.52433175
Epoch: 400 MSE: 0.5243218
Epoch: 500 MSE: 0.52432084
Epoch: 600 MSE: 0.5243211
Epoch: 700 MSE: 0.524321
Epoch: 800 MSE: 0.5243208
Epoch: 900 MSE: 0.52432054


In [25]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [30]:
# placeholder
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name='theta')
y_predict = tf.matmul(X, theta, name='predictions')
error = y_predict - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
saver = tf.train.Saver()

batch_size=100
n_batches=int(np.ceil(m/batch_size))
   
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:                              # not shown
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    print(best_theta)



ValueError: Parent directory of /tmp/my_model.ckpt doesn't exist, can't save.

In [ ]:
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z,0., name='relu')



In [ ]:
n_features = 3
X = tf.placeholder(tf.float32, shape(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name='output')